# AdaptivePrecision 4D Development & Testing Notebook

**Interactive development environment for testing AdaptivePrecision with 4D Shubert function**

This notebook provides:
- **Revise.jl integration** for automatic code reloading
- **Runtime breakdown analysis** with detailed profiling
- **Accuracy comparison** between Float64Precision and AdaptivePrecision
- **Interactive parameter tuning** for optimal performance
- **Visual analysis** of results and performance metrics

## 🔧 Development Workflow:
1. **Setup** - Load packages with Revise
2. **Baseline** - Test current implementation
3. **Iterate** - Modify code, test immediately
4. **Profile** - Analyze performance bottlenecks
5. **Compare** - Validate accuracy improvements

## 📦 Setup: Packages and Revise Integration

In [ ]:
# Essential setup for development
using Pkg
Pkg.activate("../../.")  # Activate main project

# Load Revise FIRST for automatic code reloading
using Revise

# Core packages
using Globtim
using DynamicPolynomials
using DataFrames
using BenchmarkTools
using Statistics
using LinearAlgebra

# Development and analysis tools
using ProfileView  # For performance profiling
using PlotlyJS     # For interactive plots
using Printf       # For formatted output

# Load our testing framework
include("../../test/adaptive_precision_4d_framework.jl")

println("🚀 Development environment ready!")
println("📋 Revise active - Globtim changes will auto-reload")
println("⚡ BenchmarkTools available: $BENCHMARKTOOLS_AVAILABLE")
println("📊 Framework functions loaded: $(length(TEST_FUNCTIONS_4D)) test functions")

## 🎯 Phase 2A: Shubert 4D Baseline Testing

In [ ]:
# Shubert 4D parameters (matching existing examples)
const n, a, b = 4, 2, 1
const scale_factor = a / b
const center = [0.0, 0.0, 0.0, 0.0]

# Testing parameters - start small for development
degrees_to_test = [4, 6, 8]  # Will expand as we optimize
samples_to_test = [50, 100, 200]  # Will scale up

println("🎯 Shubert 4D Testing Configuration:")
println("  Dimension: $n")
println("  Scale factor: $scale_factor")
println("  Center: $center")
println("  Degrees to test: $degrees_to_test")
println("  Samples to test: $samples_to_test")
println("  Function: shubert_4d")

### 🔍 Single Test Case - Detailed Analysis

In [ ]:
# Start with a single test case for detailed analysis
test_degree = 6
test_samples = 100

println("🔬 Detailed Analysis: Shubert 4D (degree=$test_degree, samples=$test_samples)")
println("=" ^ 70)

# Create test input
println("\n📊 Creating test input...")
@time TR = test_input(
    shubert_4d,
    dim = n,
    center = center,
    GN = test_samples,
    sample_range = scale_factor,
    degree_max = test_degree + 4
)

println("✅ Test input created with $(TR.N) samples")
println("   Sample range: $scale_factor")
println("   Center: $center")

### ⏱️ Runtime Breakdown: Construction Phase

In [ ]:
# Detailed timing breakdown for polynomial construction
println("\n⏱️  Construction Phase Runtime Breakdown:")
println("-" ^ 50)

# Float64Precision construction with detailed timing
println("\n🔍 Float64Precision Construction:")
GC.gc()  # Clean garbage collection
@time pol_float64 = Constructor(TR, test_degree, basis=:chebyshev, precision=Float64Precision, verbose=0)

# More detailed benchmark
bench_f64 = @benchmark Constructor($TR, $test_degree, basis=:chebyshev, precision=Float64Precision, verbose=0) samples=5
println("  Median time: $(median(bench_f64.times) / 1e9) seconds")
println("  Memory allocated: $(bench_f64.memory) bytes")
println("  L2 norm: $(pol_float64.nrm)")
println("  Coefficients: $(length(pol_float64.coeffs))")

# AdaptivePrecision construction with detailed timing
println("\n🚀 AdaptivePrecision Construction:")
GC.gc()  # Clean garbage collection
@time pol_adaptive = Constructor(TR, test_degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0)

# More detailed benchmark
bench_adaptive = @benchmark Constructor($TR, $test_degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0) samples=5
println("  Median time: $(median(bench_adaptive.times) / 1e9) seconds")
println("  Memory allocated: $(bench_adaptive.memory) bytes")
println("  L2 norm: $(pol_adaptive.nrm)")
println("  Coefficients: $(length(pol_adaptive.coeffs))")
println("  Precision setting: $(pol_adaptive.precision)")

# Performance comparison
overhead = median(bench_adaptive.times) / median(bench_f64.times)
memory_overhead = bench_adaptive.memory / bench_f64.memory

println("\n📊 Performance Comparison:")
println("  Time overhead: $(round(overhead, digits=3))x")
println("  Memory overhead: $(round(memory_overhead, digits=3))x")
println("  L2 norm difference: $(abs(pol_adaptive.nrm - pol_float64.nrm))")

### 🔄 Runtime Breakdown: Monomial Conversion Phase

In [ ]:
# Detailed timing for monomial conversion (where precision conversion happens)
println("\n🔄 Monomial Conversion Phase Runtime Breakdown:")
println("-" ^ 50)

@polyvar x[1:n]

# Float64Precision monomial conversion
println("\n📊 Float64Precision Monomial Conversion:")
GC.gc()
@time mono_float64 = to_exact_monomial_basis(pol_float64, variables=x)

bench_mono_f64 = @benchmark to_exact_monomial_basis($pol_float64, variables=$x) samples=3
coeffs_f64 = [coefficient(t) for t in terms(mono_float64)]

println("  Median time: $(median(bench_mono_f64.times) / 1e9) seconds")
println("  Memory allocated: $(bench_mono_f64.memory) bytes")
println("  Monomial terms: $(length(coeffs_f64))")
println("  Coefficient type: $(typeof(coeffs_f64[1]))")

# AdaptivePrecision monomial conversion (this is where BigFloat conversion happens)
println("\n🎯 AdaptivePrecision Monomial Conversion:")
GC.gc()
@time mono_adaptive = to_exact_monomial_basis(pol_adaptive, variables=x)

bench_mono_adaptive = @benchmark to_exact_monomial_basis($pol_adaptive, variables=$x) samples=3
coeffs_adaptive = [coefficient(t) for t in terms(mono_adaptive)]

println("  Median time: $(median(bench_mono_adaptive.times) / 1e9) seconds")
println("  Memory allocated: $(bench_mono_adaptive.memory) bytes")
println("  Monomial terms: $(length(coeffs_adaptive))")
println("  Coefficient type: $(typeof(coeffs_adaptive[1]))")

# Conversion phase comparison
conversion_overhead = median(bench_mono_adaptive.times) / median(bench_mono_f64.times)
conversion_memory_overhead = bench_mono_adaptive.memory / bench_mono_f64.memory

println("\n📈 Conversion Phase Comparison:")
println("  Time overhead: $(round(conversion_overhead, digits=3))x")
println("  Memory overhead: $(round(conversion_memory_overhead, digits=3))x")
println("  Terms match: $(length(coeffs_f64) == length(coeffs_adaptive))")

# Show sample BigFloat coefficients
if length(coeffs_adaptive) > 0
    println("\n🔍 Sample AdaptivePrecision Coefficients:")
    for i in 1:min(3, length(coeffs_adaptive))
        println("  $i: $(coeffs_adaptive[i])")
    end
end

### 📈 Coefficient Analysis and Sparsity

In [ ]:
# Detailed coefficient analysis
println("\n📈 Coefficient Distribution Analysis:")
println("-" ^ 50)

if length(coeffs_adaptive) > 0
    # Analyze coefficient distribution
    @time analysis = analyze_coefficient_distribution(mono_adaptive)
    
    println("\n📊 Distribution Statistics:")
    println("  Total terms: $(analysis.n_total)")
    println("  Dynamic range: $(analysis.dynamic_range)")
    println("  Max coefficient: $(analysis.max_coefficient)")
    println("  Min coefficient: $(analysis.min_coefficient)")
    println("  Suggested thresholds: $(length(analysis.suggested_thresholds))")
    
    # Test truncation at different thresholds
    println("\n✂️  Sparsity Analysis:")
    thresholds = [1e-15, 1e-12, 1e-10, 1e-8, 1e-6]
    
    sparsity_results = []
    for threshold in thresholds
        @time truncated_poly, stats = truncate_polynomial_adaptive(mono_adaptive, threshold)
        
        result = (
            threshold = threshold,
            kept = stats.n_kept,
            total = stats.n_total,
            sparsity = stats.sparsity_ratio,
            largest_removed = stats.largest_removed,
            smallest_kept = stats.smallest_kept
        )
        push!(sparsity_results, result)
        
        println("  Threshold $(threshold): keep $(stats.n_kept)/$(stats.n_total) ($(round(stats.sparsity_ratio*100, digits=1))% sparse)")
    end
    
    # Find optimal threshold
    if length(analysis.suggested_thresholds) > 0
        optimal_threshold = analysis.suggested_thresholds[1]
        println("\n🎯 Optimal Threshold Analysis ($(optimal_threshold)):")
        @time truncated_optimal, stats_optimal = truncate_polynomial_adaptive(mono_adaptive, optimal_threshold)
        
        println("  Original terms: $(stats_optimal.n_total)")
        println("  Kept terms: $(stats_optimal.n_kept)")
        println("  Sparsity achieved: $(round(stats_optimal.sparsity_ratio*100, digits=1))%")
        println("  Storage reduction: $(round((1-stats_optimal.sparsity_ratio)*100, digits=1))%")
    end
else
    println("⚠️  No coefficients available for analysis")
end

### 🔬 Performance Profiling

In [ ]:
# Profile AdaptivePrecision construction to identify bottlenecks
println("🔬 Performance Profiling - AdaptivePrecision Construction")
println("-" ^ 60)

# Profile the construction
using Profile

# Clear previous profile data
Profile.clear()

# Profile AdaptivePrecision construction
println("\n📊 Profiling AdaptivePrecision construction...")
@profile for i in 1:3
    pol = Constructor(TR, test_degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0)
end

# Display profile results
println("\n📈 Profile Results:")
Profile.print(maxdepth=10, mincount=10)

# For interactive profiling in notebook, uncomment:
# ProfileView.view()

println("\n💡 Profiling complete. Use ProfileView.view() for interactive analysis.")

## 🎯 Phase 2B: Systematic Parameter Study

In [ ]:
# Systematic study across degrees and sample sizes
println("🎯 Phase 2B: Systematic Parameter Study")
println("=" ^ 50)

# Results storage
study_results = []

for degree in degrees_to_test
    for samples in samples_to_test
        println("\n🔍 Testing: degree=$degree, samples=$samples")
        
        try
            # Create test input
            TR_study = test_input(
                shubert_4d,
                dim = n,
                center = center,
                GN = samples,
                sample_range = scale_factor,
                degree_max = degree + 4
            )
            
            # Benchmark both precisions
            bench_f64 = @benchmark Constructor($TR_study, $degree, basis=:chebyshev, precision=Float64Precision, verbose=0) samples=3
            bench_adaptive = @benchmark Constructor($TR_study, $degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0) samples=3
            
            # Construct for analysis
            pol_f64 = Constructor(TR_study, degree, basis=:chebyshev, precision=Float64Precision, verbose=0)
            pol_adaptive = Constructor(TR_study, degree, basis=:chebyshev, precision=AdaptivePrecision, verbose=0)
            
            # Store results
            result = Dict(
                :degree => degree,
                :samples => samples,
                :f64_time_median => median(bench_f64.times) / 1e9,
                :adaptive_time_median => median(bench_adaptive.times) / 1e9,
                :time_overhead => median(bench_adaptive.times) / median(bench_f64.times),
                :f64_memory => bench_f64.memory,
                :adaptive_memory => bench_adaptive.memory,
                :memory_overhead => bench_adaptive.memory / bench_f64.memory,
                :f64_norm => pol_f64.nrm,
                :adaptive_norm => pol_adaptive.nrm,
                :norm_difference => abs(pol_adaptive.nrm - pol_f64.nrm),
                :coeffs_count => length(pol_adaptive.coeffs)
            )
            
            push!(study_results, result)
            
            println("  ✅ Time overhead: $(round(result[:time_overhead], digits=3))x")
            println("  ✅ Memory overhead: $(round(result[:memory_overhead], digits=3))x")
            println("  ✅ L2 norm difference: $(result[:norm_difference])")
            
        catch e
            println("  ❌ Error: $e")
        end
    end
end

println("\n🎉 Parameter study completed! $(length(study_results)) test cases.")

### 📊 Results Analysis and Visualization

In [ ]:
# Convert results to DataFrame for analysis
if !isempty(study_results)
    results_df = DataFrame(study_results)
    
    println("📊 Parameter Study Results Summary:")
    println("=" ^ 40)
    
    # Overall statistics
    println("\n📈 Performance Statistics:")
    println("  Mean time overhead: $(round(mean(results_df.time_overhead), digits=3))x")
    println("  Median time overhead: $(round(median(results_df.time_overhead), digits=3))x")
    println("  Min time overhead: $(round(minimum(results_df.time_overhead), digits=3))x")
    println("  Max time overhead: $(round(maximum(results_df.time_overhead), digits=3))x")
    
    println("\n💾 Memory Statistics:")
    println("  Mean memory overhead: $(round(mean(results_df.memory_overhead), digits=3))x")
    println("  Median memory overhead: $(round(median(results_df.memory_overhead), digits=3))x")
    
    println("\n🎯 Accuracy Statistics:")
    println("  Mean L2 norm difference: $(mean(results_df.norm_difference))")
    println("  Max L2 norm difference: $(maximum(results_df.norm_difference))")
    println("  Cases with identical norms: $(sum(results_df.norm_difference .< 1e-15))")
    
    # Detailed breakdown by degree
    println("\n📋 Breakdown by Degree:")
    for degree in unique(results_df.degree)
        degree_data = filter(row -> row.degree == degree, results_df)
        avg_overhead = mean(degree_data.time_overhead)
        println("  Degree $degree: $(nrow(degree_data)) tests, avg overhead $(round(avg_overhead, digits=3))x")
    end
    
    # Detailed breakdown by sample size
    println("\n📊 Breakdown by Sample Size:")
    for samples in unique(results_df.samples)
        sample_data = filter(row -> row.samples == samples, results_df)
        avg_overhead = mean(sample_data.time_overhead)
        println("  Samples $samples: $(nrow(sample_data)) tests, avg overhead $(round(avg_overhead, digits=3))x")
    end
    
    # Display full results table
    println("\n📋 Complete Results Table:")
    show(results_df, allrows=true)
    
else
    println("⚠️  No results to analyze")
end

## 🔄 Phase 2C: Interactive Development Section

**This section is for iterative development and testing**

Use this space to:
- Test modifications to AdaptivePrecision implementation
- Experiment with different parameters
- Profile specific bottlenecks
- Validate accuracy improvements

**Revise.jl will automatically reload Globtim changes!**

In [ ]:
# Interactive development cell - modify as needed
println("🔄 Interactive Development Section")
println("=" ^ 40)

# Quick test after making changes to Globtim
# Revise will automatically reload changes

# Example: Test a specific configuration
test_config = (
    degree = 8,
    samples = 150,
    function_name = :shubert
)

println("🧪 Testing configuration: $test_config")

# Quick comparison
try
    result, pol_f64, pol_adaptive = compare_4d_precisions(
        test_config.function_name, 
        test_config.degree, 
        test_config.samples
    )
    
    println("✅ Test successful!")
    println("  Time overhead: $(round(result[:precision_overhead], digits=3))x")
    println("  L2 norms: F64=$(round(result[:float64_norm], digits=6)), Adaptive=$(round(result[:adaptive_norm], digits=6))")
    println("  Coefficients: $(result[:adaptive_coeffs])")
    
catch e
    println("❌ Test failed: $e")
end

println("\n💡 Modify Globtim code and re-run this cell to test changes!")

## 📋 Development Notes and Next Steps

**Use this section to track:**
- Performance bottlenecks identified
- Accuracy improvements observed
- Parameter combinations that work best
- Ideas for further optimization

### 🎯 Key Findings:
*(Update as you develop)*

### 🚀 Next Steps:
*(Update based on results)*

### 🔧 Optimization Ideas:
*(Track ideas for improvement)*